In [3]:
from loading_data import queryDB
import pandas as pd

/usr/local/lib/python3.5/dist-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [4]:
queryDB("select * from videos where filename='DocRicketts-0701_20141216T212020Z_00-48-12-26TC_h264.mp4'")

,id,filename,gpsstart,gpsstop,starttime,endtime,startdepth,enddepth,description
0,8,DocRicketts-0701_20141216T212020Z_00-48-12-26T...,"(36.798367,-122.108671)","(36.798215,-122.108195)",2014-12-16 21:20:20,2014-12-16 21:36:20,217,149,


In [5]:
annotations = queryDB('select * from annotations where videoid=8 and userid!=17 and timeinvideo>160 and timeinvideo<190')

In [6]:
annotations.head()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,433,8,6,236,178.732221,575.006251,49.006271,698.999957,155.993774,1280.0,720.0,2018-09-05,433.png,433_box.png,None,False,433,None
1,427,8,6,236,176.381214,740.006190,56.006271,842.999934,156.993767,1280.0,720.0,2018-09-05,427.png,427_box.png,None,False,427,None
2,434,8,6,236,178.485498,1116.006251,50.006271,1239.999995,156.993767,1280.0,720.0,2018-09-05,434.png,434_box.png,None,False,434,None
3,420,8,6,236,172.134367,1049.006251,203.006271,1179.993739,317.006248,1280.0,720.0,2018-09-05,420.png,420_box.png,None,False,420,None
4,422,8,6,236,172.134367,833.006251,2.006256,963.993739,116.006248,1280.0,720.0,2018-09-05,422.png,422_box.png,None,False,422,None


In [7]:
def compute_overlap(annotationA, annotationB):
    # if there is no overlap in x dimension
    if annotationB.x2 - annotationA.x1 < 0 or annotationA.x2 - annotationB.x1 < 0:
        return 0
    # if there is no overlap in y dimension
    if annotationB.y2 - annotationA.y1 < 0 or annotationA.y2 - annotationB.y1 < 0:
        return 0
    
    areaA = (annotationA.x2-annotationA.x1) * (annotationA.y2-annotationA.y1)
    areaB = (annotationB.x2-annotationB.x1) * (annotationB.y2-annotationB.y1)

    width = min(annotationA.x2,annotationB.x2) - min(annotationA.x1,annotationB.x1)
    height = min(annotationA.y2,annotationB.y2) - min(annotationA.y1,annotationB.y1)
    
    area_intersect = height * width
    iou = area_intersect / (areaA + areaB - area_intersect)
    
    return iou

In [8]:
a = annotations.iloc[0]
a.x1 = -1
a.x2 = 0
if compute_overlap(a,annotations.iloc[0]) != 0:
    print("ERROR")
    
a = annotations.iloc[0]
a.x1 = 0
a.x2 = 1000
a.y1 = 0
a.y2 = 1000
b = annotations.iloc[0]
b.x1 = 0
b.x2 = 1000
b.y1 = 0
b.y2 = 500

if compute_overlap(a,b) != .5:
    print("ERROR")

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [9]:
def score_predictions(validation,predictions,iou_thresh, concepts):
    # group predictions by video frames
    predictions = predictions.groupby(['videoid','timeinvideo'], sort=False)
    predictions = [df for _, df in predictions]
    
    # mapping annotations from prediction list to video/time
    frame_data = {}
    for i, group in enumerate(predictions):
        time = group.iloc[0]['timeinvideo']
        video = group.iloc[0]['videoid']
        frame_data[(video,time)] = i
    
    # group validation annotations by video frames
    validation = validation.groupby(['videoid','timeinvideo'], sort=False)
    validation = [df for _, df in validation]
    
    # initialize counters for each concept
    true_positives = dict(zip(concepts,[0] * len(concepts)))
    false_positives = dict(zip(concepts,[0] * len(concepts)))
    false_negatives = dict(zip(concepts,[0] * len(concepts)))
    
    # get true and false positives for each frame of validation data
    for group in validation:
        #get time and video for this set
        time = group.iloc[0]['timeinvideo']
        video = group.iloc[0]['videoid']
        
        #get corresponding predictions for this set
        predicted = predictions[frame_data[(video,time)]]
        
        #go through predictions and identify false positives/false negatives/ true_positives
        detected_predictions = []
        detected_truths = dict(zip(concepts,[0] * len(concepts)))
        for index, truth in group.iterrows():
            for index, prediction in predicted.iterrows():
                if prediction.id in detected_predictions or compute_overlap(truth,prediction) < iou_thresh or prediction.conceptid != truth.conceptid:
                    continue
                detected_predictions.append(prediction.id)
                if truth.conceptid == prediction.conceptid:
                    true_positives[prediction.conceptid] += 1
                    detected_truths[prediction.conceptid] += 1
                else:
                    print("HERE")
                    print(truth)
                    print(prediction)
                    false_positives[prediction.conceptid] += 1

        for id in concepts:
            false_negatives[id] += len(group.loc[group['conceptid'] == id]) - detected_truths[id]
        #count number of annotations that were correct, wrong, and missed
        
    f1 = dict(zip(concepts,[0] * len(concepts)))
    for id in concepts:
        recall = true_positives[id] / (true_positives[id] + false_negatives[id])
        precision = true_positives[id] / (true_positives[id] + false_positives[id])
        f1[id] = 2*recall*precision / (precision+recall)
    
    return f1

In [10]:
score_predictions(annotations,annotations,.9,annotations.conceptid.unique())

{236: 1.0, 237: 1.0, 383: 1.0}

In [11]:
#DOESNT WORK IF VAL IS MISSING

In [12]:
#is it false positive if something is misidentified, or false positive as well as false negative?
# currently both!

In [13]:
queryDB('select * from annotations limit 1')

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,2178619,19,17,2136,773.44,614.0,474.0,690.0,512.0,1280.0,720.0,2019-01-27,19_773.44_ai.png,6945_773.44_box_ai.png,None,False,6945,23180


In [14]:
queryDB('select * from annotations where userid != 17 limit 1')

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
0,11107,29,11,1083,96.110808,82.0,285.0,145.984383,321.0,1600.0,900.0,2018-11-13,11107.png,11107_box.png,,False,11107,None


In [15]:
annotations.tail()

,id,videoid,userid,conceptid,timeinvideo,x1,y1,x2,y2,videowidth,videoheight,dateannotated,image,imagewithbox,comment,unsure,originalid,framenum
53,438,8,6,236,179.579892,983.006251,87.006256,1097.999995,190.993752,1280.0,720.0,2018-09-05,438.png,438_box.png,None,False,438,None
54,462,8,6,236,189.220068,582.006251,114.006256,685.993785,210.006248,1280.0,720.0,2018-09-05,462.png,462_box.png,None,False,462,None
55,458,8,6,236,187.848553,270.006251,173.006271,379.993747,261.993752,1280.0,720.0,2018-09-05,458.png,458_box.png,None,False,458,None
56,460,8,6,236,187.848553,1191.006251,16.006256,1279.000000,98.993752,1280.0,720.0,2018-09-05,460.png,460_box.png,None,False,460,None
57,437,8,6,236,179.579892,894.006251,30.006271,1008.999995,133.993759,1280.0,720.0,2018-09-05,437.png,437_box.png,None,False,437,None


In [16]:
results = pd.read_csv('../../test/results.csv')

In [17]:
len(results.objectid.value_counts())

52

In [223]:
len(annotations)

58

In [ ]:
for 

In [78]:
results

,Unnamed: 0,x1,y1,x2,y2,label,confidence,objectid,timeinvideo
0,0,477.479370,0.000000,640.000000,128.515671,3.0,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.00
1,1,477.000000,0.000000,640.000000,129.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.03
2,2,476.000000,3.000000,639.000000,132.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.07
3,3,475.000000,8.000000,638.000000,137.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.10
4,4,472.000000,11.000000,635.000000,140.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.13
5,5,471.000000,16.000000,634.000000,145.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.17
6,6,468.000000,19.000000,631.000000,148.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.20
7,7,467.000000,24.000000,630.000000,153.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.23
8,8,464.000000,29.000000,627.000000,158.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.27
9,9,463.000000,34.000000,626.000000,163.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.30


In [85]:
label = None
objects = results.groupby(['objectid'])
for oid, result in objects:
    scores = {}
    for k , label in result.groupby(['label']):
        scores[k] = label.confidence.mean()
    idmax = max(scores.keys(), key=(lambda k: scores[k]))
    results[results.objectid == oid].label = idmax
    #print(scores[idmax])

/usr/local/lib/python3.5/dist-packages/pandas/core/generic.py:4405: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [83]:
results

,Unnamed: 0,x1,y1,x2,y2,label,confidence,objectid,timeinvideo
0,0,477.479370,0.000000,640.000000,128.515671,3.0,0.546251,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.00
1,1,477.000000,0.000000,640.000000,129.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.03
2,2,476.000000,3.000000,639.000000,132.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.07
3,3,475.000000,8.000000,638.000000,137.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.10
4,4,472.000000,11.000000,635.000000,140.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.13
5,5,471.000000,16.000000,634.000000,145.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.17
6,6,468.000000,19.000000,631.000000,148.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.20
7,7,467.000000,24.000000,630.000000,153.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.23
8,8,464.000000,29.000000,627.000000,158.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.27
9,9,463.000000,34.000000,626.000000,163.000000,NaN,NaN,d240d1ee-c37c-4fc5-90cb-d36e5e326f02,0.30


In [34]:
objects = results.groupby(['objectid']).apply((lambda x: "GI"))
for x, df in objects:
    print(x)
    print(df)
    break

objectid
046825c8-205f-4f25-a0c1-677b94b9fda4    GI
04c83311-b970-4bd7-94bd-f8c8eed5fb70    GI
05741a83-7f14-4148-8d77-c3bf9534586f    GI
05d8abb2-a36b-4d75-ab99-ffc6a728c514    GI
18cfc87d-f384-49f8-ad74-5116cae40184    GI
2b1b1eb0-79e2-4203-8f6d-48331e408136    GI
2ccb360c-af14-4f48-8242-5341419ffeb2    GI
32631bae-4451-44a7-8178-a4c6a80eadbc    GI
3798ea61-138f-41b0-b86b-8ee06c635de4    GI
380e2ef3-8f18-4962-8ec4-e72e300881c6    GI
39064ba8-34ad-4f77-8d80-3b0ec4920d56    GI
3911d0df-72df-477a-a609-5ed74dcfef97    GI
49696f21-ef18-421c-a0d4-fd06cb1651d6    GI
49dbd93b-2124-4c6b-8ff3-b37f7ae7ba41    GI
4abaf3d9-a17c-4c9a-a275-675797dd8024    GI
511e3786-3d42-4c24-abef-ed3805520d6a    GI
55eb4033-1dff-4729-8661-1aeb46560875    GI
56e017f8-037c-4d13-8d9f-6e804d40f73d    GI
653814bd-5892-458f-b487-eddc345f603b    GI
683a8e37-a9c8-4ab2-8802-3f5ed5ea0a96    GI
78bf72f6-55d9-429f-ad22-154b506b6ad1    GI
839c6a8f-444e-45fa-92f1-d2b5189de046    GI
83a8fb4c-58fc-4272-87f8-54782baa91fc    GI
85